#1. Importer DataSet


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/Projet Master/DataSets/BaseDeDonneesV3_1.zip" minutiae_detection.zip 

In [ ]:
from zipfile import ZipFile

print("Deecompression .....")
with ZipFile('minutiae_detection.zip') as zip:
    zip.extractall('Dataset')

Deecompression .....


#2. Loads libs

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Conv2D, Dense, Input, MaxPool2D, Dropout, Activation, Normalization, Flatten, Reshape
from keras.activations import relu, softmax
from keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split
import random 
from itertools import combinations
import pandas as pd
import datetime
import csv

#3. Model Creation

In [ ]:
def generat_model(optimizer=None,input_shape=(),nb_classe=2, individual=[]):
  
  my_model = Sequential(name="Model")
  my_model.add(Input(shape=input_shape))
  #my_model.add(Reshape((28,28,1)))

  conv_layers = individual[0]
  dense_layers = individual[1]
  
  for i,layer in enumerate(conv_layers):
    my_model.add(Conv2D(layer[0], layer[1], name=f'conv{i}',padding="same")) #Add conv parameters
    if layer[3] == 1:  my_model.add(Activation(relu))
    if layer[2] != 0: my_model.add(MaxPool2D((layer[2],layer[2]),padding="same"))
    

  my_model.add(Flatten())

  for i,layer in enumerate(dense_layers):
    my_model.add(Dense(layer[0]))
    if layer[1] == 1:  my_model.add(Activation(relu))

  my_model.add(Dense(nb_classe,activation=softmax))

  my_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return my_model

def generat_model_2(optimizer=None,input_shape=(), individual=[]):
  my_model = Sequential(name="Model")
  my_model.add(Input(shape=input_shape))
  #my_model.add(Reshape((28,28,1)))

  conv_layers = individual[0]
  dense_layers = individual[1]
  
  for i,layer in enumerate(conv_layers):
    my_model.add(Conv2D(layer[0], layer[1], name=f'conv{i}',padding="same")) #Add conv parameters
    if layer[3] == 1:  my_model.add(Activation(relu))
    if layer[2] != 0: my_model.add(MaxPool2D((layer[2],layer[2]),padding="same"))
    

  my_model.add(Flatten())

  for i,layer in enumerate(dense_layers):
    my_model.add(Dense(layer[0]))
    if layer[1] == 1:  my_model.add(Activation(relu))

  my_model.add(Dense(3))

  my_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return my_model

#4. Genetic Algorithme

## Search space

In [ ]:
nb_bloc_conv_valeus = [3,4,5]
nb_filters_valeus = [32,64,128,256,512] #[10,20,30,40]
filter_size_valeus = [3,5,7,9]
nb_dense_layer_valeus = [1,2,3]
nb_dense_units_valeus = [64,128,256,512]
polling_size_valeus = [0,3,5] #Dans le cas ou c'est 0 on ajoute pas de pooling

learning_rate = [0.1,0.01,0.02,0.001,0.002,0.0001] 

search_space_conv = [nb_filters_valeus,filter_size_valeus,polling_size_valeus,[0,1]]
search_space_dense = [nb_dense_units_valeus,[0,1]]


## Population generation

In [ ]:
def generate_population(population_size):

  population = []
  individual = []
  conv_layers =  []
  layer = []
  dense_layers = []
  for i in range(0,population_size):
    
    conv_layers =  []
    nb_conv_layers = random.choice(nb_bloc_conv_valeus)
    for j in range(nb_conv_layers):
      layer.append(random.choice(nb_filters_valeus))
      layer.append(random.choice(filter_size_valeus))
      layer.append(random.choice(polling_size_valeus))
      layer.append(random.choice([0,1])) #Choixe d'ajouter ou pas une couche d'activation
      conv_layers.append(layer)
      layer = []
    individual.append(conv_layers)
    
    dense_layers = []
    nb_dense_layers = random.choice(nb_dense_layer_valeus)
    for j in range(nb_dense_layers):
      layer.append(random.choice(nb_dense_units_valeus))
      layer.append(random.choice([0,1])) #Choix d'ajouter ou pas une couche d'activation
      dense_layers.append(layer)
      layer=[]
    individual.append(dense_layers)

    population.append(individual)
    individual = []

  return population

## Evaluation

In [ ]:
def fitness_function(individual,optimizer=None,input_shape=(),
                     nb_classe=2,train_set = [],
                     test_set=[],nb_epochs = 4, 
                     batch_size = 100,validation_split = 0.2):
  try:
    model = generat_model(optimizer=optimizer,input_shape=input_shape,nb_classe=nb_classe,individual=individual)
    model.fit(x = train_set[0], batch_size=batch_size, epochs=nb_epochs)
    # validation_split=validation_split,
    test_loss, test_acc = model.evaluate(test_set[0], steps=len(test_set[0]))

    print(f"test loss:{test_loss}, test accuracy:{test_acc}")
  
  except:
    return 0
    
  return test_loss,test_acc

def evaluate_population(population = [], optimizer = None,input_shape=(), train_set = [], test_set=[], nb_epochs = 15,batch_size = 50,file_path=None):
  
  evaluated_population = {}
  evaluated = []
  if len(train_set) != 0: 
    
    for i,individual in enumerate(population):
      print("Evaluation individu: ",i)
      test_loss, fitness = fitness_function(optimizer=optimizer, individual = individual,input_shape=input_shape,
                                 train_set=train_set,test_set=test_set,nb_epochs=nb_epochs,batch_size=batch_size)
      #evaluated_population[tuple(individual)] = fitness
      evaluated.append((individual,fitness))

      with open(file_path,"a") as f:
        f.write(f"{individual}:{fitness} \n")
      f.close()

  return evaluated

## Selection parents

In [ ]:
def best_ranked_selection(nb_parents, population:list):

  population_sorted = sorted(population, key=lambda x: x[1],reverse=True)
  parents  = [parent for parent in [indiv[0] for indiv in population_sorted]][0:nb_parents]
  
  return parents

def rank_selection(population, num_parents):
    fitness = [individual.fitness for individual in population]
    rank = np.argsort(np.argsort(fitness))[::-1]
    selected_parents = []
    for i in range(num_parents):
        rand = np.random.randint(0, sum(range(len(rank)))+1)
        for j in range(len(rank)):
            rand -= j
            if rand < 0:
                selected_parents.append(population[rank[j]])
                break
    return selected_parents

def random_selection(population, nb_parents):
  selected = []
  parents = []
  for i in range(nb_parents):
    r = random.randint(0,len(population))
    while r in selected:
      r = random.randint(0,len(population))
    
    selected.append(r)
    parents.append(list(population.keys())[r])
  
  return parents

## Corossover

In [ ]:
def Single_Point_Crossover(parent_1, parent_2):

  child_1,child_2 = [],[]

  min_length = min(len(parent_1[0]),len(parent_2[0]))

  random_point = random.randint(0,min_length)

  child_1.append(parent_1[0][0:random_point]+parent_2[0][random_point:len(parent_2[0])])
  child_2.append(parent_2[0][0:random_point]+parent_1[0][random_point:len(parent_1[0])])

  min_length = min(len(parent_1[1]),len(parent_2[1]))

  random_point = random.randint(0,min_length)

  child_1.append(parent_1[1][0:random_point]+parent_2[1][random_point:len(parent_2[1])])
  child_2.append(parent_2[1][0:random_point]+parent_1[1][random_point:len(parent_1[1])])

  return child_1,child_2


## Mutation

In [ ]:
def add_conv_layer(individual):
  if len(individual[1])<max(nb_bloc_conv_valeus):
    layer = []
    layer.append(random.choice(nb_filters_valeus))
    layer.append(random.choice(filter_size_valeus))
    layer.append(random.choice(polling_size_valeus))
    layer.append(random.choice([0,1]))

    rand_pos = random.randint(0,len(individual[0]))

    individual[0].insert(rand_pos,layer)
  
  return individual
  

def del_conv_layer(individual):
  if len(individual[0]) > min(nb_bloc_conv_valeus):
    rand_layer = random.randrange(0,len(individual[0]))
    del individual[0][rand_layer]

  return individual
 

def alter_conv_layer(individual):

  rand_layer = random.randrange(0,len(individual[0]))
  rand_param = random.randrange(0,len(individual[0][0]))
  val_selected = random.choice(search_space_conv[rand_param])

  while val_selected == individual[0][rand_layer][rand_param]:
    val_selected = random.choice(search_space_conv[rand_param])

  individual[0][rand_layer][rand_param] = val_selected

  return individual

def add_dense_layer(individual):

  if len(individual[1])<max(nb_dense_layer_valeus):
    layer = []
    
    layer.append(random.choice(nb_dense_units_valeus))
    layer.append(random.choice([0,1]))

    rand_pos = random.randint(0,len(individual[1]))

    individual[1].insert(rand_pos,layer)
  
  return individual
  

def del_dense_layer(individual):
  if len(individual[1]) > min(nb_dense_layer_valeus):
    rand_layer = random.randrange(0,len(individual[1]))
    del individual[1][rand_layer]

  return individual
  

def alter_dense_layer(individual):

  rand_layer = random.randrange(0,len(individual[1]))
  rand_param = random.randrange(0,len(individual[1][0]))
  val_selected = random.choice(search_space_dense[rand_param])

  while val_selected == individual[1][rand_layer][rand_param]:
    val_selected = random.choice(search_space_dense[rand_param])

  individual[1][rand_layer][rand_param] = val_selected

  return individual
  

def uniform_mutation(individual):
  
  rand_operation = random.choice([1,2,3])

  while rand_operation==2 and len(individual[1]) ==1:
     rand_operation = random.choice([1,2,3])

  if rand_operation == 1:individual = add_dense_layer(individual)
  elif rand_operation == 2:individual = del_dense_layer(individual)
  else:individual = alter_dense_layer(individual)

  rand_operation = random.choice([1,2,3])

  rand_operation = random.choice([1,2,3])
  while rand_operation==2 and len(individual[1]) ==1:
     rand_operation = random.choice([1,2,3])

  if rand_operation == 1: individual = add_conv_layer(individual)
  elif rand_operation == 2:individual = del_conv_layer(individual)
  else: individual = alter_conv_layer(individual)
  
  return individual

## select next generation

In [ ]:
def select_next_generation(population = [],children=[],next_generation_size=10, elite_frac = 0.5, children_frac = 0.5):
  
  nb_population = int(next_generation_size*elite_frac)
  nb_children = int(next_generation_size*children_frac)

  population_sorted = sorted(population, key=lambda x: x[1], reverse=True)  

  next_generation = []
  next_generation +=  [indiv[0] for indiv in population_sorted[0:nb_population]]
  next_generation += random.sample(children,nb_children)

  if (elite_frac+children_frac) < 1 :
    nb_random_indivs = int((1 - (elite_frac+children_frac)) * next_generation_size)

    for i in range(nb_random_indivs):
      random_number = random.randint(0, len(population))
      next_generation.append(population.keys()[random_number])

  return next_generation

## Select best in generation

In [ ]:
def select_best_solution(population = []):
  sorted_population = sorted(population, key=lambda x: x[1])
  return sorted_population[-1] 

## Algorithme

In [ ]:
best_in_generation = []
def Genetic_Algorithme(population_size, nb_generation,nb_parents,elite_frac,children_frac,optimizer,input_shape, train_set, test_set, nb_epochs,batch_size, file_path):
  
  #initalise la population aleatoirement
  population = generate_population(population_size)
  

  for i in range(nb_generation):
    
    with open(file_path, "a") as f:
      f.write(f"Generation:{i} \n")
    f.close()

    print("Debut generation: ",i)
    #Evaluer la population
    population_evaluated = evaluate_population(population=population,optimizer=optimizer,input_shape=input_shape, train_set=train_set,test_set= test_set, 
                                     nb_epochs=nb_epochs,batch_size=batch_size, file_path=file_path)
    
    best_in_generation.append(select_best_solution(population_evaluated))
    
    #appliquer une selection en rouleutte pour
    parents = best_ranked_selection(nb_parents,population_evaluated)
    
    #Appliquer le crossover pour cree de nouveau enfants
    new_children = []
    for parent_1, parent_2 in combinations(parents,2):
      #appliquer le crossover sur chaque 2 pers de parents
      child_1,child_2 = Single_Point_Crossover(parent_1,parent_2)
      new_children.append(child_1)
      new_children.append(child_2)

    #Appliquer la mutation sur les enfants
    children_after_mutation = []
    for child in new_children:
      mutated_child = uniform_mutation(child)
      children_after_mutation.append(mutated_child)
    
    #selection la population de la future generation
    population = select_next_generation(population_evaluated, children_after_mutation,population_size,elite_frac,children_frac)
    """
    #Evaluer la population
    population_evaluated = evaluate_population(population=population,optimizer=optimizer,input_shape=input_shape, train_set=train_set,test_set= test_set, 
                                     nb_epochs=nb_epochs,batch_size=batch_size,file_path)"""

    
  
  
  #Selectionner la meilleur solution attiente
  best_solution = select_best_solution(population_evaluated)

  return best_solution

#4. Datas

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        validation_split=0.2)

training_set= train_datagen.flow_from_directory(
        '/content/Dataset/BaseDeDonneesV3_1/TrainSet',
        target_size=(32, 32),
        color_mode="grayscale",
        batch_size=150,
        class_mode='categorical'
        )


test_datagen = ImageDataGenerator(
        rescale=1./255)

test_set= test_datagen.flow_from_directory(
        '/content/Dataset/BaseDeDonneesV3_1/TestSet',
        target_size=(32, 32),
        color_mode="grayscale",
        batch_size=150,
        class_mode='categorical'
        )



Found 60000 images belonging to 2 classes.
Found 17121 images belonging to 2 classes.


# Test

In [ ]:
NB_OF_GENERATION = 10
POPULATION_SIZE = 15
NB_PARENTS = 10
INPUT_SHAPE =(32,32,1)
BATCH_SIZE = 100
NB_EPOCHS = 8
ELITE_FRAC = 0.5
CHILDREN_FRAC = 0.5
TEST_SIZE = 0.4

In [ ]:
#train_setX, test_setX, train_setY, test_setY = load_dataset("/content/Dataset/train.csv",TEST_SIZE)

In [ ]:
date = datetime.datetime.now()
FILE_NAME = f"file_{date}.txt"
f = open(FILE_NAME,"w")
f.write(f"NB_OF_GENERATION = {NB_OF_GENERATION}\nPOPULATION_SIZE = {POPULATION_SIZE}\nINPUT_SHAPE = {INPUT_SHAPE}\nBATCH_SIZE = {BATCH_SIZE}\nNB_EPOCHS = {NB_EPOCHS}\nELITE_FRAC = {ELITE_FRAC}\nCHILDREN_FRAC = {CHILDREN_FRAC}\nTEST_SIZE = {TEST_SIZE}\n")
for best in best_in_generation: 
  print(best)
  f.write(str(best)+"\n")

f.close()


In [ ]:
optimizer = SGD(0.02)
best_solution = Genetic_Algorithme(POPULATION_SIZE,NB_OF_GENERATION,NB_PARENTS,ELITE_FRAC,CHILDREN_FRAC,optimizer,
                  INPUT_SHAPE,[training_set],[test_set],NB_EPOCHS,BATCH_SIZE, FILE_NAME)
print("Best Solution: ",best_solution)

Debut generation:  0
Evaluation individu:  0
Epoch 1/8
400/400 [==============================] - 37s 68ms/step - loss: 0.6463 - accuracy: 0.6608
Epoch 2/8
400/400 [==============================] - 27s 66ms/step - loss: 0.5686 - accuracy: 0.6827
Epoch 3/8
400/400 [==============================] - 26s 65ms/step - loss: 0.5430 - accuracy: 0.7001
Epoch 4/8
400/400 [==============================] - 26s 64ms/step - loss: 0.5204 - accuracy: 0.7205
Epoch 5/8
400/400 [==============================] - 26s 65ms/step - loss: 0.4859 - accuracy: 0.7557
Epoch 6/8
400/400 [==============================] - 26s 64ms/step - loss: 0.4515 - accuracy: 0.7854
Epoch 7/8
400/400 [==============================] - 26s 65ms/step - loss: 0.4168 - accuracy: 0.8065
Epoch 8/8
115/115 [==============================] - 7s 55ms/step - loss: 0.3432 - accuracy: 0.8532
test loss:0.34320709109306335, test accuracy:0.8531627655029297
Evaluation individu:  1
Epoch 1/8
400/400 [==============================] - 26s 62m

In [ ]:
!cp -r "file_{date}.txt" "/content/drive/MyDrive/Colab Notebooks/Projet Master/Tests/file_{date}.txt"